## Constants


In [4]:
from pathlib import Path

PATH_TO_MODEL_DIR = Path('models/research/object_detection/inference_graph')
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR / "saved_model"
PATH_TO_LABELS = Path('label_map.pbtxt')

PATH_TO_IMAGES = Path('data/train eval 1/eval/images')


## Imports


In [5]:
import random
from object_detection.utils import ops as utils_ops
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import tensorflow as tf
import os

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings


## Load Saved Inference Model


In [9]:
print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(str(PATH_TO_SAVED_MODEL))

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))


Loading model...Done! Took 21.018110990524292 seconds


In [10]:
category_index = label_map_util.create_category_index_from_labelmap(str(PATH_TO_LABELS),
                                                                    use_display_name=True)


In [13]:
image_paths = [PATH_TO_IMAGES /
               file for file in os.listdir(PATH_TO_IMAGES) if file.endswith('.jpg')]

print([img.name for img in image_paths[:10]])


['IMG_1718_00108006.jpg', 'IMG_1718_00108017.jpg', 'IMG_1724_00057604.jpg', 'IMG_1724_00057753.jpg', 'IMG_1724_00057763.jpg', 'IMG_1726_00057640.jpg', 'IMG_1726_00057645.jpg', 'IMG_1727_00108005.jpg', 'IMG_1727_00108100.jpg', 'IMG_1729_00108521.jpg']


In [14]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


In [23]:
from cv2 import imshow as cv2_imshow


for image_path in [random.choice(image_paths)]:

    print("Running inference for : ", image_path)
    image_np = load_image_into_numpy_array(image_path)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop("num_detections"))

    # detections = {key: value[0, :num_detections].numpy()
    #             for key, value in detections.items()}

    import itertools
    detections = dict(itertools.islice(detections.items(), num_detections))

    detections["num_detections"] = num_detections

    # detection_classes should be ints.
    # detections["detection_classes"] = detections["detection_classes"].astype(np.int64)

    image_np_with_detections = image_np.copy()

    # Handle models with masks:
    if "detection_masks" in detections:
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detections["detection_masks"][0], detections["detection_boxes"][0],
            image_np.shape[0], image_np.shape[1])
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                           tf.uint8)
        detections["detection_masks_reframed"] = detection_masks_reframed.numpy()

    boxes = np.asarray(detections["detection_boxes"][0])
    classes = np.asarray(detections["detection_classes"][0]).astype(np.int64)
    scores = np.asarray(detections["detection_scores"][0])
    mask = np.asarray(detections["detection_masks_reframed"])

    # Visualizing the results
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        boxes,
        classes,
        scores,
        category_index,
        instance_masks=mask,
        use_normalized_coordinates=True,
        skip_boxes=True,
        skip_scores=True,
        skip_labels=True,
        line_thickness=3)

    viz_utils.save_image_array_as_png(
        image_np_with_detections, "test_image.jpg")

    # cv2_imshow(image_np_with_detections)
    # display(Image.fromarray(image_np_with_detections))
    print("Done")


Running inference for :  data\train eval 1\eval\images\IMG_1741_00108115.jpg
Done
